In [1]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

session = SnorkelSession()
train_cands = session.query(Spouse).filter(Spouse.split == 0).all()
dev_cands = session.query(Spouse).filter(Spouse.split == 1).all()
test_cands = session.query(Spouse).filter(Spouse.split == 2).all()


# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]
gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
import numpy as np
unique, counts = np.unique(gold_labels_dev, return_counts=True)
print(dict(zip(unique, counts)))


unique, counts = np.unique(gold_labels_test, return_counts=True)
print(dict(zip(unique, counts)))

print(len(gold_labels_dev),len(gold_labels_test))

{0: 2625, 1: 189}
{0: 2484, 1: 218}
2814 2702


In [5]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]
    
output shape : [NoOfDataPoints,2,NoOfLFs]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [3]:

# spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}

#drop 1
spouses = {'spouse', 'ex-wife', 'ex-husband'}



# family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
#               'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}

#drop 1
family = {'sister', 'son', 'brother',
              'grandfather', 'grandmother', 'aunt', 'cousin'}

family = family | {f + '-in-law' for f in family}
# other = {'boyfriend', 'girlfriend', 'boss', 'employee', 'secretary', 'co-worker'}

#drop 1
other = {'boyfriend', 'girlfriend', 'boss', 'secretary'}

In [7]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

In [4]:
####### Discrete ##########




# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_husband_wife_left_window(c):
    if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (1,1)
    elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (1,1)
    else:
        return (0,0)
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
def LF_familial_relationship(c):
    return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

def LF_family_left_window(c):
    if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
        return (-1,1)
    elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
        return (-1,1)
    else:
        return (0,0)

def LF_other_relationship(c):
    return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


import bz2

# Function to remove special characters from text
def strip_special(s):
    return ''.join(c for c in s if ord(c) < 128)

# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
    known_spouses = set(
        tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [6]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_discrete_drop1",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_discrete_drop1",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_discrete_drop1",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 28-6-2018, 10:0:16
500data points labelled in 0.0679487943649292 mins
1000data points labelled in 0.12447514533996581 mins
1500data points labelled in 0.16112714608510334 mins
2000data points labelled in 0.20181631247202556 mins
2500data points labelled in 0.2369531234105428 mins
500data points labelled in 0.31459869543711344 mins
1000data points labelled in 0.3620237708091736 mins
1500data points labelled in 0.413549800713857 mins
2000data points labelled in 0.45886727174123126 mins
2500data points labelled in 0.5014333724975586 mins
3000data points labelled in 0.5369452118873597 mins
3500data points labelled in 0.5766034801801045 mins
4000data points labelled in 0.60789981285731 mins
4500data points labelled in 0.6417740821838379 mins
5000data points labelled in 0.6851561625798543 mins
5500data points labelled in 0.720976710319519 mins
6000data points labelled in 0.7562177101771037 mins
6500data points labelled in 0.7916414062182109 mins
7000data points labelled in 0.8287

In [8]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    write_to_file(wordvec_unavailable)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [9]:
##### Continuous ################





# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for sw in spouses:
        sc=max(sc,get_similarity(word_vectors,sw))
    return (1,sc)

def LF_husband_wife_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for sw in spouses:
        sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for sw in spouses:
        sc_2=max(sc_2,get_similarity(word_vectors,sw))
    return(1,max(sc_1,sc_2))
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    global LF_Threshold
    word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
    sc = get_similarity(word_vectors,'married')
    
    if 'and' in get_between_tokens(c):
        return (1,sc)
    else:
        return (0,0)

def LF_familial_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for fw in family:
        sc=max(sc,get_similarity(word_vectors,fw))
        
    return (-1,sc) 

def LF_family_left_window(c):
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for fw in family:
        sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for fw in family:
        sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
    return (-1,max(sc_1,sc_2))

def LF_other_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
        
    return (-1,sc) 

# def LF_other_relationship_left_window(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
#     for ow in other:
#         sc=max(sc,get_similarity(word_vectors,ow))
#     return (-1,sc) 

import bz2

# Function to remove special characters from text
def strip_special(s):
    s = s.decode("utf-8") 
    return ''.join(c for c in s if (ord(c) < 128))


# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'r') as f:
    known_spouses = set(
        tuple(strip_special(x).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

import numpy as np

# def LF_Three_Lists_Left_Window(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife_left_window(c)
#     c2,s2 = LF_family_left_window(c)
#     c3,s3 = LF_other_relationship_left_window(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'Left:',s1,s2,s3,prob_sc
    
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1

# def LF_Three_Lists_Between_Words(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife(c)
#     c2,s2 = LF_familial_relationship(c)
#     c3,s3 = LF_other_relationship(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
    
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'BW:',s1,s2,s3,prob_sc
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1
    
LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [10]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_smooth_drop1",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_smooth_drop1",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_smooth_drop1",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 28-6-2018, 10:5:59
500data points labelled in 0.4450263778368632 mins
1000data points labelled in 0.8385651866594951 mins
1500data points labelled in 1.2300942103068033 mins
2000data points labelled in 1.619545817375183 mins
2500data points labelled in 2.0547518571217855 mins
500data points labelled in 2.713345197836558 mins
1000data points labelled in 3.1187922517458597 mins
1500data points labelled in 3.6051185448964436 mins
2000data points labelled in 3.9978747328122455 mins
2500data points labelled in 4.428113301595052 mins
3000data points labelled in 4.943234308560689 mins
3500data points labelled in 5.360162055492401 mins
4000data points labelled in 5.744744598865509 mins
4500data points labelled in 6.091235323746999 mins
5000data points labelled in 6.4737404425938925 mins
5500data points labelled in 6.884695561726888 mins
6000data points labelled in 7.2674635688463844 mins
6500data points labelled in 7.644230151176453 mins
7000data points labelled in 8.06182601451873

In [11]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import average_precision_score

In [25]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]
import numpy as np
dev_L_S = np.load("dev_L_S_smooth_drop1.npy")
test_L_S = np.load("test_L_S_smooth_drop1.npy")
train_L_S = np.load("train_L_S_smooth_drop1.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")
# true_labels = gold_labels_test
print(len(gold_labels_dev),len(gold_labels_test))
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

2814 2702
(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)


In [16]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]

dev_L_S = np.load("dev_L_S_discrete_drop1.npy")
test_L_S = np.load("test_L_S_discrete_drop1.npy")
train_L_S = np.load("train_L_S_discrete_drop1.npy")
print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)


In [20]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [12]:
import scipy.sparse as sp
import _pickle as pkl


L_train = np.load("train_L_S_discrete_drop1.npy")
L_train = L_train[:,0,:].astype(int)
print(np.array(L_train).shape)
L_train = sp.csr_matrix(L_train)


from snorkel.learning import GenerativeModel
import time
import datetime

gen_model = GenerativeModel()

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, epochs = 100)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)


print("trained in ",str(datetime.timedelta(seconds=time.time() - start_time)))

(22276, 10)
started at: 28-6-2018, 10:37:44
Inferred cardinality: 2
trained in  0:01:15.761812


In [13]:

L_dev = np.load("dev_L_S_discrete_drop1.npy")
L_dev = L_dev[:,0,:].astype(int)
print(np.array(L_dev).shape)
L_dev = sp.csr_matrix(L_dev)

marginals = gen_model.marginals(L_dev)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_dev),GenLabels,average="binary"))


(2814, 10)
(2814,)
(2814,)
(0.4375, 0.037037037037037035, 0.06829268292682926, None)


In [14]:

L_test = np.load("test_L_S_discrete_drop1.npy")
L_test = L_test[:,0,:].astype(int)
print(np.array(L_test).shape)
L_test = sp.csr_matrix(L_test)

marginals = gen_model.marginals(L_test)
marginals = np.array(marginals)
print(marginals.shape)

# GenLabels = np.argmax(dev_marginals,axis=1)
GenLabels =  np.array([1 if m > 0.5 else 0 for m in marginals])
print(GenLabels.shape)

print(precision_recall_fscore_support(np.array(gold_labels_test),GenLabels,average="binary"))


(2702, 10)
(2702,)
(2702,)
(0.3333333333333333, 0.022935779816513763, 0.04291845493562232, None)


In [17]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

10


In [18]:

def train(lr,ep,th,af,batch_size=32,LF_acc=None,LF_rec=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        test_init_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)
        
        

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        if(penalty in [4,5,7,8,9,10,11]):
            LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(penalty in [6,7,11,12]):
            LF_r = tf.convert_to_tensor(LF_rec, dtype=tf.float64)
        
        if(debug):
            print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        if(debug):
            print("s",s)
            print("l",l)
#         print(s.graph)
        if(smooth):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
            if(debug):
                print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        if(debug):
            print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        if(debug):
            print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            if(debug):
                print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            if(debug):
                print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")
            
        ### for precision and recall t_pout
        def p_t_pout(j):
            Lj = tf.map_fn(lambda li : tf.gather(li,j),l)
            if(debug):
                print("sft Lj",Lj)
            kj = tf.gather(k,j)
            if(debug):
                print("sft kj",kj)
            indices = tf.where(tf.equal(Lj,kj))
            if(debug):
                print("sft indices",indices)
            li_lij_eq_kj = tf.gather(l,tf.squeeze(indices,1))
            if(smooth):
                si_lij_eq_kj = tf.gather(s_,tf.squeeze(indices,1))
            if(debug):
                print("sft l_ij_eq_kj",li_lij_eq_kj)
            if(smooth):
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                prec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                prec_t_pout = (tf.matmul(li_lij_eq_kj*si_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            else:
                prec_t_pout = (tf.matmul(li_lij_eq_kj*kj, thetas,transpose_b=True))/prec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return prec_t_pout
        
        def r_t_pout(j):
            kj = tf.gather(k,j)
            if(debug):
                print("r_t_pout kj",kj)

            if(smooth):
                rec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="prec_zy"))
            else:
                rec_z = tf.reduce_sum(tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="prec_zy"))
            if(debug):
                print("prec_z",prec_z)
            if(smooth):
                rec_t_pout = (tf.matmul(l*s_*kj, thetas,transpose_b=True))/rec_z
            else:
                rec_t_pout = (tf.matmul(l*kj, thetas,transpose_b=True))/rec_z
            if(debug):
                print("prec_t_pout",prec_t_pout)
            return rec_t_pout
            
           
        def softplus_p(j):
            aj = tf.gather(LF_a,j)
            if(debug):
                print("sft aj",aj)
            f_p =  tf.reduce_sum(aj - p_t_pout(j))
            if(debug):
                print("f_p",f_p)
            sft_p = tf.nn.softplus(f_p,name="sft_p")
            if(debug):
                print("sft_p",sft_p)
            return sft_p
        
       
        def softplus_r(j):
            rj = tf.gather(LF_r,j)
            if(debug):
                print("sft aj",rj)
            
            f_r =  rj - tf.reduce_sum(r_t_pout(j))
            if(debug):
                print("f_r",f_r)
            sft_r = tf.nn.softplus(f_r,name="sft_r")
            if(debug):
                print("sft_r",sft_r)
            return sft_r
        
              
#         logsft = tf.map_fn(lambda j: tf.log(softplus(j)),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
#         sft  =  tf.map_fn(lambda j: softplus(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)
        
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        if(debug):
            print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        if(debug):
            print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        if(debug):
            print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 4):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus_p(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 6):
            print("recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 7):
            print("precision and recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 8):
            print("precision and sign 1 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 9):
            print("precision and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 10):
            print("precision and sign 3 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 11):
            print("precision and recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 12):
            print("recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))

        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
       
        if(debug):
            print("loss",loss)
        tf.summary.scalar('loss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)
                print("dev set")
                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
                print()
                print("test set")
                sess.run(test_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_test,pl))
                print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary"))
                print()
                
#             # Initialize an iterator over the validation dataset.
#             sess.run(dev_init_op)
#             a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
#             print(t)

#             unique, counts = np.unique(pl, return_counts=True)
#             print(dict(zip(unique, counts)))

#             print("acc",accuracy_score(true_labels,pl))

# #             predictAndPrint(pl)
#             print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [21]:
train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(0,0.001,seed),\
                                pcl=np.array([-1,1],dtype=np.float64),\
                                norm=True,smooth=False)

k Tensor("Const:0", shape=(10,), dtype=float64)
s Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normalized loss
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 178620.6270051208
dev set
[ 1.17544629e-03 -1.85785468e-03  9.55965364e-05  6.24306244e-06
  1.06377870e-03  4.13385488e-04  1.69205997e-03  3.12078490e-04
  1.46334713e-03  6.47232968e-04]
[[1.01873877 0.71621435 0.910700

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [26]:
for i in np.linspace(0.6,1,5):
    print("alpha-init:",i)
    train(0.1/len(train_L_S),5,batch_size = 1, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=0,debug=False)

alpha-init: 0.6
normalized loss
0 loss 150446.6538828145
dev set
[0.67895034 0.64872874 0.66510822 0.6586544  0.65278212 0.51754202
 0.64311814 0.5117279  0.51015653 0.51257962]
[[1.0427522  0.77059971 0.94827742 0.94291863 1.06300952 1.12495884
  1.10663918 1.09778858 1.20797462 1.13436817]]
{0: 2808, 1: 6}
acc 0.9328358208955224
(0.5, 0.015873015873015872, 0.03076923076923077, None)

test set
{0: 2696, 1: 6}
acc 0.9178386380458919
(0.16666666666666666, 0.0045871559633027525, 0.00892857142857143, None)

1 loss 143919.392630721
dev set
[0.65218303 0.6334208  0.71238985 0.70053367 0.65862724 0.43570028
 0.65993987 0.41406463 0.41200091 0.41500893]
[[1.02802618 0.76635184 0.90776834 0.90657365 1.04820292 1.20836018
  1.08379325 1.17727439 1.28747463 1.21635349]]
{0: 2813, 1: 1}
acc 0.9331911869225302
(1.0, 0.005291005291005291, 0.010526315789473682, None)

test set
{0: 2701, 1: 1}
acc 0.9189489267209474
(0.0, 0.0, 0.0, None)

2 loss 135407.9002453128
dev set
[0.57181151 0.58605797 0.7445

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


3 loss 125978.76945522723
dev set
[0.47848041 0.51786778 0.76403741 0.74594441 0.60363531 0.28355063
 0.64913161 0.21487888 0.21292493 0.21585174]
[[1.13275743 0.81442961 0.92798626 0.91510999 1.0551342  1.36813945
  1.11846424 1.3564586  1.46890773 1.39910898]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

test set
{0: 2702}
acc 0.9193190229459659
(0.0, 0.0, 0.0, None)

4 loss 116464.40582355422
dev set
[0.38077958 0.43597585 0.76577295 0.75604401 0.54888084 0.21787016
 0.63209871 0.11542772 0.11352846 0.11641755]
[[1.21322566 0.8630629  0.9895416  0.96679113 1.07551691 1.44236765
  1.15915676 1.44917211 1.56245991 1.49318476]]
{0: 2814}
acc 0.9328358208955224
(0.0, 0.0, 0.0, None)

test set
{0: 2702}
acc 0.9193190229459659
(0.0, 0.0, 0.0, None)

alpha-init: 0.7
normalized loss
0 loss 153185.94989852078
dev set
[0.77455266 0.73752674 0.74531107 0.7413194  0.74354237 0.62422006
 0.74521289 0.63832567 0.65085467 0.63976573]
[[1.05245279 0.78440909 0.95378512 0.94501519 1.07400

1 loss 154240.58704640408
dev set
[1.23195657 0.98673101 1.23183308 1.         1.23195997 1.2320527
 1.23195197 1.23204831 1.23201831 1.23204695]
[[1.35319978 0.829161   1.24458345 1.00062431 1.34199526 1.27678032
  1.40503605 1.26659874 1.38214032 1.30025461]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

2 loss 153644.66672899295
dev set
[1.33617434 0.98402877 1.33609344 1.         1.3361815  1.33632764
 1.33615344 1.33632754 1.33625497 1.33631255]
[[1.45866125 0.83417334 1.35006032 1.00062431 1.44745777 1.38227437
  1.51049319 1.37209436 1.48762256 1.40574553]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

3 loss 152091

In [27]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.85)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.85*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=2,debug=False)

[18934.6 18934.6 18934.6 18934.6 18934.6 18934.6 18934.6 18934.6 18934.6
 18934.6]
batch-size: 128 alpha-init: 0.6
penalty2
0 loss 152971.4657883507
dev set
[0.60195729 0.59892333 0.60087737 0.60079238 0.60184422 0.59963073
 0.60247835 0.59952998 0.60068249 0.5998663 ]
[[1.11676377 0.81343613 1.0087766  0.99983628 1.10560065 1.04212149
  1.16841652 1.03198721 1.14711142 1.06548788]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

1 loss 152927.06805325072
dev set
[0.60273339 0.59969797 0.60165475 0.60156488 0.60262009 0.59884766
 0.60324653 0.59874562 0.5998985  0.59908301]
[[1.11598808 0.81266349 1.00800186 0.99906252 1.1048267  1.04290577
  1.16764061 1.03275906 1.14787375 1.06626088]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set


3 loss 154398.51785977095
dev set
[0.90412871 0.89974566 0.90313152 0.90305991 0.90235204 0.90351294
 0.90475174 0.89787759 0.89921125 0.90129347]
[[1.11476545 0.81387066 1.00660047 0.99762463 1.10652099 1.03826189
  1.16618921 1.03364722 1.14865829 1.06499926]]
{0: 2740, 1: 74}
acc 0.9214641080312722
(0.28378378378378377, 0.1111111111111111, 0.1596958174904943, None)

test set
{0: 2640, 1: 62}
acc 0.9119170984455959
(0.3387096774193548, 0.0963302752293578, 0.15, None)

4 loss 154398.08709384763
dev set
[0.90485177 0.90013192 0.90387372 0.90380629 0.90266267 0.90427716
 0.90550011 0.89727223 0.89865841 0.90145529]
[[1.1140916  0.81380108 1.00588176 0.99689636 1.10657229 1.0375102
  1.16545712 1.03425565 1.14923272 1.0650675 ]]
{0: 2740, 1: 74}
acc 0.9214641080312722
(0.28378378378378377, 0.1111111111111111, 0.1596958174904943, None)

test set
{0: 2640, 1: 62}
acc 0.9119170984455959
(0.3387096774193548, 0.0963302752293578, 0.15, None)

batch-size: 128 alpha-init: 1.0
penalty2
0 loss 154

batch-size: 256 alpha-init: 0.8
penalty2
0 loss 154321.6320400497
dev set
[0.80156735 0.79852435 0.80048941 0.80040099 0.80144892 0.80001909
 0.8020904  0.79992094 0.80108052 0.80026947]
[[1.11715468 0.81385321 1.00916677 1.00023034 1.10601002 1.04173275
  1.16881122 1.03158731 1.14670528 1.06492061]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0.13970588235294118, None)

1 loss 154319.2662580987
dev set
[0.80195638 0.79890553 0.80087907 0.8007912  0.80183221 0.79962858
 0.80248091 0.799542   0.80071172 0.79989893]
[[1.11676725 0.81349002 1.00877792 0.99984079 1.1056415  1.04212388
  1.16842138 1.03195241 1.14705744 1.06511601]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0.13970

{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

2 loss 152965.71641867977
dev set
[0.60176466 0.59873143 0.60068535 0.60059637 0.60165291 0.59982311
 0.60228267 0.59972171 0.60087364 0.60005714]
[[1.11695561 0.81362557 1.00896911 1.00003328 1.10578938 1.04192901
  1.16861506 1.03179811 1.1469224  1.06531095]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

3 loss 152954.52255139724
dev set
[0.60196051 0.59892721 0.60088185 0.60079222 0.60184881 0.59962612
 0.60247826 0.59952452 0.60067655 0.59985994]
[[1.11675977 0.81342982 1.008773   0.99983739 1.10559359 1.04212618
  1.16841914 1.03199441 1.14711737 1.06550764]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1:

penalty2
0 loss 154405.46569398642
dev set
[1.00137947 0.99797639 1.00032022 1.000036   1.00126843 1.00062655
 1.00189418 1.00052849 1.00166616 1.00085578]
[[1.11768804 0.81438369 1.00956152 1.00062431 1.10650897 1.04136608
  1.1693828  1.03122316 1.14650066 1.06478942]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

1 loss 154405.46563686457
dev set
[1.00160806 0.99779889 1.00064107 1.00035683 1.00149996 1.00089523
 1.00211373 1.00080981 1.00188902 1.00110501]
[[1.11789563 0.81456968 1.0095938  1.00062663 1.10671107 1.04146895
  1.16959972 1.03130097 1.14671564 1.06494417]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

2 l

2 loss 154321.3149058821
dev set
[0.8014704  0.79843576 0.80039087 0.80030172 0.80135811 0.80011782
 0.80198829 0.80001699 0.80116977 0.80035337]
[[1.11724998 0.81392476 1.00926447 1.00032893 1.10608632 1.0416342
  1.16891063 1.03150007 1.14662383 1.06495251]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0.13970588235294118, None)

3 loss 154320.70990965978
dev set
[0.80156859 0.79853355 0.8004891  0.80039999 0.8014561  0.80001934
 0.80208668 0.79991917 0.80107263 0.80025562]
[[1.11715188 0.81382814 1.00916629 1.0002307  1.10598921 1.04173282
  1.16881242 1.03159675 1.14671938 1.06503006]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0.13970588235294118, None)

4 loss 154320.104545

0 loss 153915.2560402274
dev set
[0.70122477 0.69819147 0.70014506 0.70005578 0.70111307 0.70036399
 0.70174145 0.70026255 0.70141372 0.70059782]
[[1.11749532 0.81416525 1.00951023 1.00057485 1.10632864 1.04138794
  1.16915658 1.03125742 1.14638427 1.06477239]]
{0: 2751, 1: 63}
acc 0.9211087420042644
(0.23809523809523808, 0.07936507936507936, 0.11904761904761904, None)

test set
{0: 2650, 1: 52}
acc 0.9126572908956329
(0.3269230769230769, 0.0779816513761468, 0.1259259259259259, None)

1 loss 153914.18615079703
dev set
[0.70127401 0.69824076 0.7001944  0.70010519 0.70116234 0.70031472
 0.70179066 0.7002132  0.70136445 0.70054855]
[[1.11744609 0.81411595 1.00946085 1.00052545 1.10627941 1.04143722
  1.16910728 1.0313067  1.14643335 1.06482151]]
{0: 2751, 1: 63}
acc 0.9211087420042644
(0.23809523809523808, 0.07936507936507936, 0.11904761904761904, None)

test set
{0: 2650, 1: 52}
acc 0.9126572908956329
(0.3269230769230769, 0.0779816513761468, 0.1259259259259259, None)

2 loss 153913.11766

2 loss 154405.46568279553
dev set
[1.00132645 0.99799735 1.00025932 1.00008802 1.00121508 1.0005689
 1.00184216 1.00046933 1.00161378 1.00080042]
[[1.11767841 0.81436012 1.00956248 1.00062433 1.10650672 1.04138509
  1.16935037 1.03123522 1.14647606 1.06481261]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

3 loss 154405.4656680052
dev set
[1.00137941 0.99794856 1.00032661 1.00015666 1.00126838 1.00062699
 1.00189407 1.00052931 1.00166607 1.00085589]
[[1.11772968 0.81441004 1.00956727 1.00062457 1.10655741 1.0414151
  1.16940234 1.03125665 1.14652796 1.06485592]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

4 loss 154405.4

In [28]:
acc = np.empty([len(LF_l)],dtype=np.float64)
acc.fill(0.25)
rec =  np.empty([len(LF_l)],dtype=np.float64)
rec.fill(0.35*train_L_S.shape[0])
print(rec)
### smooth LFs with acc on discrete LFs
for b in [128,256,512,1024,2048]:
    for i in np.linspace(0.6,1,5):
        print("batch-size:",b,"alpha-init:",i)
        train(0.1/len(train_L_S),5,batch_size = b, th = tf.truncated_normal_initializer(1,0.1,seed),\
                                    af = tf.truncated_normal_initializer(i,0.001,seed),\
                                    LF_acc = acc ,LF_rec = rec,\
                                    pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=6,debug=False)

[7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6 7796.6]
batch-size: 128 alpha-init: 0.6
recall penalty
0 loss 13797030.551050954
dev set
[0.60195728 0.59892331 0.60087736 0.60079237 0.60184422 0.59963073
 0.60247834 0.59952999 0.6006825  0.5998663 ]
[[1.11676378 0.81343616 1.0087766  0.99983627 1.10560065 1.04212148
  1.16841654 1.03198719 1.14711138 1.06548786]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

1 loss 13796986.239447234
dev set
[0.60273338 0.59969794 0.60165475 0.60156487 0.60262009 0.59884766
 0.60324653 0.59874563 0.59989851 0.59908301]
[[1.1159881  0.81266355 1.00800185 0.99906251 1.10482671 1.04290576
  1.16764063 1.03275902 1.14787368 1.06626084]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 

2 loss 13798449.171716196
dev set
[0.90339856 0.89934131 0.90238276 0.90230716 0.90201694 0.90274486
 0.90399718 0.89848486 0.89976766 0.9011513 ]
[[1.11544963 0.81395756 1.00732809 0.99836146 1.10649131 1.03901998
  1.1669298  1.03303913 1.14808362 1.06491698]]
{0: 2740, 1: 74}
acc 0.9214641080312722
(0.28378378378378377, 0.1111111111111111, 0.1596958174904943, None)

test set
{0: 2640, 1: 62}
acc 0.9119170984455959
(0.3387096774193548, 0.0963302752293578, 0.15, None)

3 loss 13798448.738730557
dev set
[0.90412757 0.8997298  0.90313101 0.90305964 0.90232553 0.90351288
 0.90475147 0.89788327 0.89921649 0.90131806]
[[1.1147681  0.81388777 1.00660171 0.99762536 1.10654645 1.03826194
  1.1661899  1.03364455 1.14865581 1.06498119]]
{0: 2740, 1: 74}
acc 0.9214641080312722
(0.28378378378378377, 0.1111111111111111, 0.1596958174904943, None)

test set
{0: 2640, 1: 62}
acc 0.9119170984455959
(0.3387096774193548, 0.0963302752293578, 0.15, None)

4 loss 13798448.311000317
dev set
[0.90485035 0.90

4 loss 7014890.838395191
dev set
[0.70312379 0.70008204 0.7020429  0.70195199 0.70300732 0.69844833
 0.70363248 0.69835666 0.69953651 0.69870246]
[[1.11559868 0.81228991 1.00761074 0.99867293 1.10444718 1.04330752
  1.16725434 1.03312422 1.14820566 1.06661948]]
{0: 2756, 1: 58}
acc 0.9221748400852878
(0.2413793103448276, 0.07407407407407407, 0.11336032388663966, None)

test set
{0: 2653, 1: 49}
acc 0.9137675795706883
(0.3469387755102041, 0.0779816513761468, 0.12734082397003746, None)

batch-size: 256 alpha-init: 0.8
recall penalty
0 loss 7015331.775464965
dev set
[0.80156734 0.79852424 0.80048941 0.80040098 0.80144885 0.80001911
 0.80209036 0.79992098 0.80108062 0.80026956]
[[1.11715471 0.8138535  1.00916678 1.00023035 1.10601029 1.04173271
  1.16881123 1.03158725 1.14670518 1.06491912]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0

1 loss 3583489.9589943346
dev set
[0.60156863 0.59853541 0.60048899 0.60040056 0.60145684 0.60001984
 0.60208701 0.5999186  0.60107031 0.600254  ]
[[1.11715161 0.81382156 1.00916536 1.00022936 1.10598532 1.04173216
  1.1688112  1.03160201 1.14672761 1.06511467]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

2 loss 3583478.8127029887
dev set
[0.60176466 0.59873143 0.60068535 0.60059637 0.60165291 0.59982311
 0.60228267 0.59972171 0.60087364 0.60005714]
[[1.11695561 0.81362557 1.00896911 1.00003328 1.10578938 1.04192901
  1.16861506 1.0317981  1.14692239 1.06531094]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

3 loss 35834

4 loss 3584903.720890682
dev set
[0.90214273 0.89889549 0.9010712  0.90098379 0.90171342 0.90139441
 0.90266893 0.89940655 0.90059803 0.90101258]
[[1.1165933  0.81401105 1.00859113 0.99965109 1.10649306 1.0403597
  1.16823314 1.03211475 1.14721522 1.0648964 ]]
{0: 2740, 1: 74}
acc 0.9214641080312722
(0.28378378378378377, 0.1111111111111111, 0.1596958174904943, None)

test set
{0: 2640, 1: 62}
acc 0.9119170984455959
(0.3387096774193548, 0.0963302752293578, 0.15, None)

batch-size: 512 alpha-init: 1.0
recall penalty
0 loss 3584909.4654173115
dev set
[1.00137947 0.99797595 1.00032022 1.00003599 1.00126843 1.00062655
 1.00189418 1.00052849 1.00166616 1.00085578]
[[1.11768804 0.81438418 1.00956152 1.00062431 1.10650897 1.04136608
  1.1693828  1.03122316 1.14650066 1.06478942]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.066945606694

2 loss 1869575.4599405208
dev set
[0.8014704  0.79843574 0.80039087 0.80030172 0.8013581  0.80011783
 0.80198828 0.800017   0.8011698  0.80035339]
[[1.11724999 0.81392482 1.00926448 1.00032894 1.10608636 1.0416342
  1.16891063 1.03150005 1.1466238  1.06495155]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0.13970588235294118, None)

3 loss 1869574.858197409
dev set
[0.80156859 0.79853353 0.80048909 0.80039999 0.80145609 0.80001935
 0.80208667 0.79991918 0.80107266 0.80025564]
[[1.11715188 0.81382821 1.00916629 1.0002307  1.10598927 1.04173281
  1.16881242 1.03159673 1.14671934 1.06502877]]
{0: 2740, 1: 74}
acc 0.9221748400852878
(0.2972972972972973, 0.1164021164021164, 0.16730038022813687, None)

test set
{0: 2648, 1: 54}
acc 0.9133974833456698
(0.35185185185185186, 0.0871559633027523, 0.13970588235294118, None)

4 loss 1869574.25608

4 loss 1010616.0592424063
dev set
[0.60142175 0.59838856 0.60034214 0.60025286 0.60131014 0.60016695
 0.60193846 0.60006563 0.60121697 0.60040082]
[[1.11729835 0.81396808 1.00931291 1.00037738 1.10613154 1.04158501
  1.16895944 1.03145466 1.14658134 1.06496957]]
{0: 2758, 1: 56}
acc 0.9221748400852878
(0.23214285714285715, 0.06878306878306878, 0.10612244897959183, None)

test set
{0: 2660, 1: 42}
acc 0.9156180606957809
(0.38095238095238093, 0.07339449541284404, 0.12307692307692308, None)

batch-size: 2048 alpha-init: 0.7
recall penalty
0 loss 1011546.2897173876
dev set
[0.70122477 0.69819147 0.70014506 0.70005578 0.70111307 0.70036399
 0.70174145 0.70026255 0.70141372 0.70059782]
[[1.11749532 0.81416525 1.00951023 1.00057485 1.10632864 1.04138794
  1.16915658 1.03125742 1.14638427 1.06477239]]
{0: 2751, 1: 63}
acc 0.9211087420042644
(0.23809523809523808, 0.07936507936507936, 0.11904761904761904, None)

test set
{0: 2650, 1: 52}
acc 0.9126572908956329
(0.3269230769230769, 0.077981651376

2 loss 1012031.4653999826
dev set
[1.00132645 0.99799751 1.00025932 1.00008802 1.00121508 1.0005689
 1.00184216 1.00046933 1.00161378 1.00080042]
[[1.11767841 0.81436002 1.00956248 1.00062433 1.10650672 1.04138509
  1.16935037 1.03123522 1.14647606 1.06481261]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

3 loss 1012031.4653783225
dev set
[1.00137941 0.99794907 1.00032661 1.00015666 1.00126838 1.00062699
 1.00189407 1.00052931 1.00166607 1.00085589]
[[1.11772968 0.81440959 1.00956727 1.00062457 1.10655741 1.0414151
  1.16940234 1.03125665 1.14652796 1.06485592]]
{0: 2796, 1: 18}
acc 0.9314143567874911
(0.3888888888888889, 0.037037037037037035, 0.06763285024154589, None)

test set
{0: 2681, 1: 21}
acc 0.9174685418208735
(0.38095238095238093, 0.03669724770642202, 0.06694560669456068, None)

4 loss 1012031

In [34]:
#majority vote #84 # Regex
# train_L_S_df.mode(axis=1)[0].tolist()
import math
# test_L_S_df.mode(axis=1)[0].tolist()
test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
# test_L_S_df = test_L_S_df.astype(int)
test_L_S_df = test_L_S_df.replace(0, np.NaN)

predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()

predicted_labels = [ int(x) if not math.isnan(x) else 1 for x in predicted_labels ]
predicted_labels = [ 0 if x == -1 else 1 for x in predicted_labels ]
# print(predicted_labels)
# Results = predictAndPrint(predicted_labels)
print("acc",accuracy_score(gold_labels_test,predicted_labels))
print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(predicted_labels),average="binary"))
           

acc 0.6902294596595114
(0.17386722866174922, 0.7568807339449541, 0.2827763496143959, None)


In [35]:
#majority vote #84 # Regex
# train_L_S_df.mode(axis=1)[0].tolist()
import math
# test_L_S_df.mode(axis=1)[0].tolist()
test_L_S_df = pd.DataFrame((np.array(test_L_S)[:,0,:]))
# test_L_S_df = test_L_S_df.astype(int)
test_L_S_df = test_L_S_df.replace(0, np.NaN)

predicted_labels=test_L_S_df.mode(axis=1)[0].tolist()

predicted_labels = [ int(x) if not math.isnan(x) else -1 for x in predicted_labels ]
predicted_labels = [ 0 if x == -1 else 1 for x in predicted_labels ]
# print(predicted_labels)
# Results = predictAndPrint(predicted_labels)
print("acc",accuracy_score(gold_labels_test,predicted_labels))
print(precision_recall_fscore_support(np.array(gold_labels_test),np.array(predicted_labels),average="binary"))
           

acc 0.8915618060695781
(0.39669421487603307, 0.6605504587155964, 0.49569707401032703, None)
